# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [41]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [83]:
city_weather = pd.read_csv('../WeatherPy/output_data/cityweather.csv')

city_weather.set_index('City', inplace = True)

city_weather.drop(columns = ['Unnamed: 0'], inplace=True)

city_weather.reset_index()



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Blair,11.6667,92.7500,82.832,68,0,4.84,IN,1612062480
1,Lorengau,-2.0226,147.2712,82.580,82,99,5.30,PG,1612062480
2,Ushuaia,-54.8000,-68.3000,46.400,71,20,3.09,AR,1612062480
3,Barrow,71.2906,-156.7887,-16.600,69,90,2.06,US,1612062481
4,Ngunguru,-35.6167,174.5000,73.004,55,0,4.02,NZ,1612062481
...,...,...,...,...,...,...,...,...,...
595,Tigil',57.8000,158.6667,3.002,92,99,4.21,RU,1612062641
596,Una,20.8167,71.0333,64.832,28,0,4.98,IN,1612062641
597,Burē,10.7000,37.0667,55.688,63,8,1.98,ET,1612062641
598,Yuzhne,46.6221,31.1013,33.800,100,0,6.00,UA,1612062642


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [84]:
gmaps.configure(api_key = g_key)

In [85]:
locations = city_weather[['Lat', 'Lng']].astype(float)

weight = city_weather['Humidity'].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating = False, max_intensity = 100, point_radius = 2.5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [86]:
city_weather = city_weather.rename(columns = {
    'Max Temp': 'Temp',
    'Wind Speed': 'Wind'
})
city_weather

city_weather[city_weather.eval('Temp>=60 & Temp<90 & Humidity<35 & Cloudiness<=25 & Wind<=30')]

city_weather


,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date
City,,,,,,,,
Kidal,18.4411,1.4078,62.150,26,2,4.14,ML,1612062483
Banamba,13.5500,-7.4500,65.552,24,4,4.10,ML,1612062504
Diapaga,12.0708,1.7889,69.872,21,16,3.62,BF,1612062492
Guider,9.9342,13.9486,69.782,23,0,2.62,CM,1612062527
Tezu,27.9167,96.1667,68.090,26,24,0.69,IN,1612062556
Hadejia,12.4498,10.0444,63.878,27,0,3.93,NG,1612062587
Mapimí,25.8167,-103.8500,73.400,21,1,0.89,MX,1612062614
Tessalit,20.1986,1.0114,60.314,25,0,4.36,ML,1612062628
Una,20.8167,71.0333,64.832,28,0,4.98,IN,1612062641


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [87]:
hotel_df = city_weather[city_weather.eval('Temp>=60 & Temp<90 & Humidity<35 & Cloudiness<=25 & Wind<=30')]
hotel_df.reset_index(inplace=True)

hotel_df["Hotel"] = ""
hotel_df.head(10)

<ipython-input-87-70831105631c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel"] = ""


,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date,Hotel
0,Kidal,18.4411,1.4078,62.150,26,2,4.14,ML,1612062483,
1,Banamba,13.5500,-7.4500,65.552,24,4,4.10,ML,1612062504,
2,Diapaga,12.0708,1.7889,69.872,21,16,3.62,BF,1612062492,
3,Guider,9.9342,13.9486,69.782,23,0,2.62,CM,1612062527,
4,Tezu,27.9167,96.1667,68.090,26,24,0.69,IN,1612062556,
5,Hadejia,12.4498,10.0444,63.878,27,0,3.93,NG,1612062587,
6,Mapimí,25.8167,-103.8500,73.400,21,1,0.89,MX,1612062614,
7,Tessalit,20.1986,1.0114,60.314,25,0,4.36,ML,1612062628,
8,Una,20.8167,71.0333,64.832,28,0,4.98,IN,1612062641,


In [88]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# parameters for API call
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    # adding the lat and lng from each row, adding to params
    params["location"] = f'{lat},{lng}'
    
    print(f'Retrieving results for Index {index}: {city}')
    
    response = requests.get(base_url, params = params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel"] = results[0]['name']
    except:
        pass
    print('--------------')
    
    hotel_df.dropna(how = 'any', inplace = True)
    hotel_df

Retrieving results for Index 0: Kidal
--------------
Retrieving results for Index 1: Banamba
--------------
Retrieving results for Index 2: Diapaga


<ipython-input-88-41d823ffeb24>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how = 'any', inplace = True)
<ipython-input-88-41d823ffeb24>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how = 'any', inplace = True)


--------------
Retrieving results for Index 3: Guider
--------------
Retrieving results for Index 4: Tezu


<ipython-input-88-41d823ffeb24>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how = 'any', inplace = True)
<ipython-input-88-41d823ffeb24>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how = 'any', inplace = True)


--------------
Retrieving results for Index 5: Hadejia
--------------
Retrieving results for Index 6: Mapimí


<ipython-input-88-41d823ffeb24>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how = 'any', inplace = True)
<ipython-input-88-41d823ffeb24>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how = 'any', inplace = True)


--------------
Retrieving results for Index 7: Tessalit
--------------
Retrieving results for Index 8: Una


<ipython-input-88-41d823ffeb24>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how = 'any', inplace = True)
<ipython-input-88-41d823ffeb24>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how = 'any', inplace = True)


--------------


<ipython-input-88-41d823ffeb24>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how = 'any', inplace = True)


In [89]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [90]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

'''figure_layout = {
    'width': '450px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)'''

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))